<a name="top"></a><img src="source/SpinalHDL.png" alt="SpinalHDL based on Scala" style="width:320px;" />

运行SpinalHDL代码之前，务必加载SpinalHDL的库 
（**注**：*第一次运行时需要下载库，可能会有点慢，请耐心等待*）

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-spinal.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

### 1. 位宽推断的使用限制 
关于没有位宽声明的getWidth用法
```scala
  val c = UInt()
  c.getWidth   // X Failed !
  c:= a + b
  c.getWidth   // is ok, because c width already inferred by "c := a + b"
```
另外一个例子， setAll 函数需要getWidth方法
```scala
 val c = out UInt() 
 when(clc){c.setAll }.otherwise(c := a * b)  //failed
 when(clc){c := a * b }.otherwise(c.setAll)  //It's OK
```

In [ ]:
class T2  extends Component{
    val a,b = in UInt(8 bits)
    val clc= in Bool()
    val c = out UInt() 
//  when(clc){c.setAll }.otherwise(c := a * b)  //failed   
  when(clc){c := a * b }.otherwise(c.setAll)  //It's OK
}
showRtl(new T2)

### 2. asBool问题 
对于1bit的数据操作，第一种和后面两个方法是有区别的，请注意
- a.asBits  
- a.lsb     
- a(0) 
```scala
val a= Bits(1 bits)
a.asBits //返回一个新的Bool，将 tmp := a.lsb
a.lsb    //直接操作a(0) bit
a(0)     //
```

In [ ]:
class T1  extends Component{  
  val sel = in Bool()
  val a = Reg(Bits(1 bits)) init 0 
  when(sel){a.asBool.set()} //generate verilog beyond your expectations
}
showRtl(new T1)  

In [ ]:
class T2  extends Component{  
  val sel = in Bool()
  val a = Reg(Bits(1 bits)) init 0 
  when(sel){a(0).set()} 
  // when(sel){a.lsb.set()}  //also Ok
}
showRtl(new T2) 